# classification_models
*   arboles de decision


# Preparacion Librerías

In [ ]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt

# Preprocesado y modelado
# ------------------------------------------------------------------------------
# sklearn : scikit learn (paquete de ciencia de datos)
from sklearn.model_selection import train_test_split # párticion de datos
from sklearn.tree import DecisionTreeClassifier # modelo classifier
from sklearn.tree import plot_tree # grafica de modelo
from sklearn.tree import export_graphviz # grafica de modelo
from sklearn.tree import export_text # grafica de modelo
from sklearn.model_selection import GridSearchCV # tuneo de hiper parametros 
from sklearn.compose import ColumnTransformer # transformacion de variablees
from sklearn.preprocessing import OneHotEncoder # transformacion de variablees dummies
from sklearn.metrics import accuracy_score # metrica aciertos/totalobservaciones
from sklearn.metrics import confusion_matrix # matriz con evaluacion de predicciones

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

## problema

El set de datos Carseats, original del paquete de R ISLR y accesible en Python a través de statsmodels.datasets.get_rdataset, contiene información sobre la venta de sillas infantiles en 400 tiendas distintas. Para cada una de las 400 tiendas se han registrado 11 variables. Se pretende generar un modelo de clasificación que permita predecir si una tienda tiene ventas altas (Sales > 8) o bajas (Sales <= 8) en función de todas las variables disponibles.

## data

In [ ]:
carseats = sm.datasets.get_rdataset("Carseats", "ISLR")
datos = carseats.data # cargamos los datos en objeto datos
print(carseats.__doc__)

In [ ]:
datos.head(3)

In [ ]:
datos.info()

In [ ]:
# Como Sales es una variable continua y el objetivo del estudio es clasificar las tiendas según si venden mucho o poco, se crea una nueva variable dicotómica (0, 1) llamada ventas_altas.

datos['ventas_altas'] = np.where(datos.Sales > 8, 1, 0) # np.where ifelse de numpy
# Una vez creada la nueva variable respuesta se descarta la original
datos = datos.drop(columns = 'Sales')

In [ ]:
datos.head()

In [ ]:
datos['ventas_altas'].value_counts() # contar por clases 1 y 0

# Ajuste del modelo

In [ ]:
# datos para entrenamiento : TRAIN
# datos para validacion : TEST


# División de los datos en train y test
# ------------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
                                        datos.drop(columns = 'ventas_altas'), # las variables independientes o X
                                        datos['ventas_altas'], # la variable que quiero predecir o la Y
                                        random_state = 123 # semillas aleatorias 
                                    )

In [ ]:
# modelado de procesamiento de datos

# One-hot-encoding de las variables categóricas
# ------------------------------------------------------------------------------
# Se identifica el nobre de las columnas numéricas y categóricas
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.to_list()
numeric_cols = X_train.select_dtypes(include=['float64', 'int']).columns.to_list()

# Se indica la acción a aplicar :  one-hot-encoding solo a las columnas categóricas
preprocessor = ColumnTransformer(
                    [('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
                    remainder='passthrough'
               )

# Una vez que se ha definido el objeto ColumnTransformer, con el método fit()
# se aprenden las transformaciones con los datos de entrenamiento y se aplican a
# los dos conjuntos con transform(). Ambas operaciones a la vez con fit_transform().
X_train_prep = preprocessor.fit_transform(X_train) # ajuste y transforme/aplique transformacion
X_test_prep  = preprocessor.transform(X_test) # aplique transformacion

In [ ]:
# Convertir el output del ColumnTransformer en dataframe y añadir el nombre de las columnas
# ------------------------------------------------------------------------------
# Nombre de todas las columnas
encoded_cat = preprocessor.named_transformers_['onehot'].get_feature_names_out(cat_cols)
labels = np.concatenate([numeric_cols, encoded_cat])

# Conversión a dataframe
X_train_prep = pd.DataFrame(X_train_prep, columns=labels)
X_test_prep  = pd.DataFrame(X_test_prep, columns=labels)
X_train_prep.info()

In [ ]:
# Creación del modelo
# ------------------------------------------------------------------------------
modelo = DecisionTreeClassifier(
    # hiperparametros
            max_depth         = 5,  # maxima profundidad
            criterion         = 'gini', # criterio
            random_state      = 123 #semilla aleatoria para uso academico
          )

# Entrenamiento del modelo
# ------------------------------------------------------------------------------
modelo.fit(X_train_prep, y_train) # aqui se entrenó el modelo con los datos de entrenamiento

In [ ]:
# Estructura del árbol creado
# ------------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(13, 6))

print(f"Profundidad del árbol: {modelo.get_depth()}")
print(f"Número de nodos terminales: {modelo.get_n_leaves()}")

plot = plot_tree(
            decision_tree = modelo,
            feature_names = labels.tolist(),
            class_names   = 'ventas_altas',
            filled        = True,
            impurity      = False,
            fontsize      = 7,
            ax            = ax
       )

In [ ]:
# 0 : no es ventas altas (186)
# 1 : es ventas altas (114)
unos = 114
total = 186+114
w = unos/total
w

## Predicción y evaluación del modelo

Se evalúa la capacidad predictiva del árbol inicial calculando el accuracy en el conjunto de test.

In [ ]:
# Pruning (const complexity pruning) por validación cruzada
# ------------------------------------------------------------------------------
# Error de test del modelo
#-------------------------------------------------------------------------------
predicciones = modelo.predict(X = X_test_prep,) # obtener predicciones

print("Matriz de confusión")
print("-------------------")
confusion_matrix(
    y_true    = y_test,
    y_pred    = predicciones
)

In [ ]:
aciertos = 44 + 27
total = 44+27+23+6
accuracy =  aciertos /total
accuracy*100

In [ ]:
# El modelo inicial es capaz de predecir correctamente un % de las observaciones del conjunto de test.

accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = predicciones,
            normalize = True
           )
print(f"El accuracy de test es: {100 * accuracy} %")

## Podado del árbol (pruning

In [ ]:
# Post pruning (const complexity pruning) por validación cruzada
# ------------------------------------------------------------------------------
# Valores de ccp_alpha evaluados
param_grid = {'ccp_alpha':np.linspace(0, 5, 10)} # parametro complejidad

# Búsqueda por validación cruzada
grid = GridSearchCV( # tunear/ buscar combinación de hiperparametros 
        # El árbol se crece al máximo posible antes de aplicar el pruning
        estimator = DecisionTreeClassifier( # modelo claisficador/ regresor
                            max_depth         = None,
                            min_samples_split = 2,
                            min_samples_leaf  = 1,
                            random_state      = 123
                       ),
        param_grid = param_grid,
        scoring    = 'accuracy',
        cv         = 10, # validacion cruzada 
        refit      = True,
        return_train_score = True
      )

grid.fit(X_train_prep, y_train) # ajustamos el modelo con las indicaciones de busqueda de parametros

fig, ax = plt.subplots(figsize=(12, 3.84)) # ajustar tamaño del visort del grafico
scores = pd.DataFrame(grid.cv_results_)
scores.plot(x='param_ccp_alpha', y='mean_train_score', yerr='std_train_score', ax=ax)
scores.plot(x='param_ccp_alpha', y='mean_test_score', yerr='std_test_score', ax=ax)
ax.set_title("Error de validacion cruzada vs hiperparámetro ccp_alpha");

In [ ]:
# Mejor valor ccp_alpha encontrado
# ------------------------------------------------------------------------------
grid.best_params_

In [ ]:
# Estructura del árbol final
# ------------------------------------------------------------------------------
modelo_final = grid.best_estimator_  # seleccione el el mejor modelo con los parametros optimos 
print(f"Profundidad del árbol: {modelo_final.get_depth()}")
print(f"Número de nodos terminales: {modelo_final.get_n_leaves()}")

In [ ]:
# Error de test del modelo final
#-------------------------------------------------------------------------------
predicciones = modelo_final.predict(X = X_test_prep)

accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = predicciones,
            normalize = True
           )
print(f"El accuracy de test es: {100 * accuracy} %")

## Importancia de predictores

In [ ]:
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores = pd.DataFrame(
                            {'predictor': labels.tolist(),
                             'importancia': modelo_final.feature_importances_}
                            )
importancia_predictores.sort_values('importancia', ascending=False)

## Predicción de probabilidades

In [ ]:
# Predicción de probabilidades
#-------------------------------------------------------------------------------
predicciones = modelo.predict_proba(X = X_test_prep)
predicciones[:5, 1:] # quedarse con las dos probabilidad p(0) y p(1) es redundante suele usarse solo p(1)

In [ ]:
# Clasificación empleando la clase de mayor probabilidad
# ------------------------------------------------------------------------------
df_predicciones = pd.DataFrame(data=predicciones, columns=['0', '1'])
df_predicciones['clasificacion_default_0.5'] = np.where(df_predicciones['0'] > df_predicciones['1'], 0, 1)
df_predicciones.head(3)

In [ ]:
# Clasificación final empleando un threshold de 0.8 para la clase 1.
# ------------------------------------------------------------------------------
df_predicciones['clasificacion_custom_0.8'] = np.where(df_predicciones['1'] > 0.8, 1, 0) # np.where son el ifelse de numpy para utilizar threshold : umbral = 0.8
df_predicciones.iloc[5:10, 1:]

In [ ]:
# Clasificación final empleando un threshold de 0.8 para la clase 1.
# ------------------------------------------------------------------------------
df_predicciones['clasificacion_fal'] = np.where(df_predicciones['1'] > 0, 1, 0) # np.where son el ifelse de numpy para utilizar threshold 
df_predicciones.iloc[5:10, 1:]

## Bibliografía


* https://www.cienciadedatos.net/documentos/py07_arboles_decision_python.html